In [ ]:
#!pip install ipyleaflet

In [ ]:
import pandas as pd
import requests
from ipyleaflet import Map, Polyline
import folium
import polyline
from IPython.display import display, HTML


from folium.features import DivIcon

def number_DivIcon(number):
    """ Create a 'numbered' icon
    
    """
    icon = DivIcon(
            icon_size=(150,36),
            icon_anchor=(14,40),
#             html='<div style="font-size: 18pt; align:center, color : black">' + '{:02d}'.format(num+1) + '</div>',
            html="""<span class="fa-stack " style="font-size: 12pt" >>
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color : {:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">
                         {:02d}  
                    </strong>
                </span>""".format('#440154',number)
        )
    return icon


In [ ]:
OD_low_conf_routes = pd.read_table("../notebooks/data/Fleet Opt Low - tours.txt")
OD_high_conf_routes = pd.read_table("../notebooks/data/Fleet Opt High - tours.txt")

#### lets cross check the constraints file they used
OD_constraints = pd.read_csv("../notebooks/data/fleet_entry_exit 2.csv")

In [ ]:
def baditem(x):
    try:
        int(x)
        return False
    except:
        print(x)
        return True
    

In [ ]:
base_url = 'http://prd-tpe-placematcher.api.chrazure.cloud'

def get_lat(city, state):
    response = requests.get(f'{ base_url }/api/cities/exactish/{ city }%2C%20{ state }', verify=False)
    # print(response.json())
    lat = response.json()['lat']
    lng = response.json()['lng']
    return lat

def get_lng(city, state):
    response = requests.get(f'{ base_url }/api/cities/exactish/{ city }%2C%20{ state }', verify=False)
    # print(response.json())
    lat = response.json()['lat']
    lng = response.json()['lng']
    return lng

### geocode using zips package.

In [ ]:
bad_ix = [baditem(x) for x in list(OD_low_conf_routes["Load ID"].values)]

OD_low_conf_routes[bad_ix]

OD_low_conf_routes = OD_low_conf_routes[[b == False for b in bad_ix]]

bad_ix = [baditem(x) for x in list(OD_high_conf_routes["Load ID"].values)]

OD_high_conf_routes[bad_ix]

OD_high_conf_routes = OD_high_conf_routes[[b == False for b in bad_ix]]

OD_low_conf_routes["Load ID"] = OD_low_conf_routes["Load ID"].astype(int)
OD_high_conf_routes["Load ID"] = OD_high_conf_routes["Load ID"].astype(int)

In [ ]:
### try sorting index here to see if we can fix problems
OD_low_conf_routes = OD_low_conf_routes.sort_values(["Tour ID","Driver Start Move Date","Load Dropoff Date"]).reset_index(drop=True)

OD_high_conf_routes = OD_high_conf_routes.sort_values(["Tour ID","Driver Start Move Date","Load Dropoff Date"]).reset_index(drop=True)


In [ ]:
OD_constraints

In [ ]:
tours = OD_low_conf_routes["Tour ID"].unique()

In [ ]:
print(f""" 
there are {len(tours)} tours in this data set
""")

In [ ]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
#### ok in talking with andrew a bunch of the tours have rough edges
#### cut those to that in the center of the tour
#### this is about POC anyways
#### lets grab a few and figure it out from there
tours_data = OD_low_conf_routes

In [ ]:
#### first cut out any singlet tours
screen = tours_data.groupby(["Tour ID"]).aggregate({"Driver ID":"count"}).reset_index()
screen

In [ ]:
tours_data = tours_data[~tours_data['Tour ID'].isin(screen[screen["Driver ID"] <= 1]['Tour ID'])] ### pull singlets

In [ ]:
#tours_data = tours_data.merge(data, how = "left", left_on = "Load ID", right_on = "LOAD_NUM")

In [ ]:
### a bit more screening on where date ranges make sense to program constraints

#### get the window included in the screening
start = pd.to_datetime("1-4-2023").date()
end = pd.to_datetime("1-29-2023").date()

#### Ok we can chunk these off now and map em out
program_type = list()
tour_id = list()
start_diff = list()
end_diff = list()
window_start = list()
window_end = list()
for k,v in tours_data.groupby("Tour ID"):
    cycle = tours_data[tours_data["Tour ID"] == k]
    cycle = cycle.sort_index()

    driver_constraints = constraint_data[constraint_data["DRIVER_NAME"] == cycle.iloc[0]["Driver Name"]]
    date_match = pd.to_datetime(cycle["Driver Start Move Date"].iloc[0]).date()
    driver_constraints["tour_start_difference"] = date_match - driver_constraints["ACTIVE_DRIVER_IN_PROGRAM"]
    driver_constraints["tour_start_difference_absolute"] =  abs(driver_constraints["tour_start_difference"])
    driver_constraints = driver_constraints.sort_values("tour_start_difference_absolute")

    window_start.append(driver_constraints["ACTIVE_DRIVER_IN_PROGRAM"].iloc[0] - start)  ### want positive or zero
    
    
    date_match = pd.to_datetime(cycle["Load Dropoff Date"].iloc[-1]).date()
    driver_constraints["tour_end_difference"] = date_match - driver_constraints["DUE_HOME_ON"]
    driver_constraints["tour_end_difference_absolute"] =  abs(driver_constraints["tour_end_difference"])

    window_end.append(end - driver_constraints["DUE_HOME_ON"].iloc[0])  ### want positive or zero

    tour_id.append(k)
    start_diff.append(driver_constraints["tour_start_difference"].iloc[0])
    end_diff.append(driver_constraints["tour_end_difference"].iloc[0])
    
    program_type.append(driver_constraints["Program"].iloc[0])
    




screen = pd.DataFrame(list(zip(tour_id, start_diff,end_diff,program_type,window_start,window_end)),
               columns =['Tour', 'start diff',"end diff","Program Type","Gap to Window Start","Gap to Window End"])
screen

screen.groupby("Program Type").count()

#screen = screen[(screen["start diff"]<= datetime.timedelta(days =0)) & (screen["end diff"]<= datetime.timedelta(days =0)) ]

screen = screen[(screen["start diff"]>= datetime.timedelta(days =0)) & (screen["end diff"]<= datetime.timedelta(days =0)) ]

screen = screen[(screen["Gap to Window Start"]>= datetime.timedelta(days =0)) & (screen["Gap to Window End"]>= datetime.timedelta(days =0)) ]

tours_data = tours_data[tours_data['Tour ID'].isin(screen["Tour"])] ### pull singlets

tours = tours_data["Tour ID"].unique()

In [ ]:
print(f""" 
there are {len(tours)} usable tours in this data set
""")

In [ ]:
constraint_data[constraint_data.Program == constraint_data.Program.unique()[1]]

In [ ]:
#### Ok we can chunk these off now and map em out
#tour_id = 0
Loop_output = list()
loop_size = len(tours)
iters = range(loop_size)

### just do a specific one ABNusxzo-003
# tours = ["ABNusxzo-003"]
# loop_size = len(tours)
# iters = range(loop_size)

for tour_id in iters:
    iteration_dict = dict()
    print(f"Running {tour_id} in Routes")
    cycle = tours_data[tours_data["Tour ID"] == tours[tour_id]]

    cycle = cycle.sort_index()
    
    try:
        ### we need to make the finish the domicile.  this isn't always done 
        ### i have no clue why but we have them ending on loads a long ways away

        cycle.iloc[-1,cycle.columns.get_loc('Driver Ending City')] = cycle.iloc[0]["Driver Starting City"]




        cycle['origin_lat'] = cycle.apply(lambda row: get_lat(row['ORIGIN_CITY'], row['ORIGIN_STATE']), axis=1)
        cycle['origin_lng'] = cycle.apply(lambda row: get_lng(row['ORIGIN_CITY'], row['ORIGIN_STATE']), axis=1)
        cycle['destination_lat'] = cycle.apply(lambda row: get_lat(row['DESTINATION_CITY'], row['DESTINATION_STATE']), axis=1)
        cycle['destination_lng'] = cycle.apply(lambda row: get_lng(row['DESTINATION_CITY'], row['DESTINATION_STATE']), axis=1)


        #### do the start finish provided by OD #####
        cycle['driver_start_lat'] = cycle.apply(lambda row: get_lat(row["Driver Starting City"
                                                                       ].split(",")[0], row["Driver Starting City"
                                                                                           ].split(",")[1]), axis=1)
        cycle['driver_start_lng'] = cycle.apply(lambda row: get_lng(row["Driver Starting City"
                                                                       ].split(",")[0], row["Driver Starting City"
                                                                                           ].split(",")[1]), axis=1)

        cycle['driver_end_lat'] = cycle.apply(lambda row: get_lat(row["Driver Ending City"
                                                                       ].split(",")[0], row["Driver Ending City"
                                                                                           ].split(",")[1]), axis=1)
        cycle['driver_end_lng'] = cycle.apply(lambda row: get_lng(row["Driver Ending City"
                                                                       ].split(",")[0], row["Driver Ending City"
                                                                                           ].split(",")[1]), axis=1)


        ### get the routes

        plotting = list()
        route_keys = list()
        deadhead_start = list()
        deadhead_end = list()
        for k,row in cycle.iterrows():
            osrm_results = get_o_to_d(row['origin_lat'], row['origin_lng'], row['destination_lat'], row['destination_lng'])
            encoded_polyline = osrm_results["routes"][0]["geometry"]

            routes = polyline.decode(encoded_polyline)
            start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
            end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
            distance = osrm_results['routes'][0]['distance']

            route = {'route':routes,
                       'start_point':start_point,
                       'end_point':end_point,
                       'distance':(distance/1609.344)
                      }
            plotting.append(route)

            #### deadhead lines for starting city
            osrm_results = get_o_to_d(row['driver_start_lat'], row['driver_start_lng'], row['origin_lat'], row['origin_lng'])
            encoded_polyline = osrm_results["routes"][0]["geometry"]

            routes = polyline.decode(encoded_polyline)
            start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
            end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
            distance = osrm_results['routes'][0]['distance']

            route = {'route':routes,
                       'start_point':start_point,
                       'end_point':end_point,
                       'distance':(distance/1609.344)
                      }
            deadhead_start.append(route)


            #### deadhead lines for ending city
            osrm_results = get_o_to_d(row['destination_lat'], row['destination_lng'], row['driver_end_lat'], row['driver_end_lng'])
            encoded_polyline = osrm_results["routes"][0]["geometry"]

            routes = polyline.decode(encoded_polyline)
            start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
            end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
            distance = osrm_results['routes'][0]['distance']

            route = {'route':routes,
                       'start_point':start_point,
                       'end_point':end_point,
                       'distance': (distance/1609.344)
                      }
            deadhead_end.append(route)


            route_keys.append(k)



        cycle["Routes"] = plotting
        cycle["Deadhead_Start"] = deadhead_start
        cycle["Deadhead_End"] = deadhead_end

        ### make the f-ing map
        m_od = folium.Map()
        num = 1
        box=list()
        for k,row in cycle.iterrows():
            popup_pick = f"""L{row["LOAD_NUM"]} <br>
            Pick Ready: {row["PICK_READY_DATE"].strftime("%m/%d")}
            Pick By: {row["PICK_BY_DATE"].strftime("%m/%d")}
            """

            popup_drop = f"""L{row["LOAD_NUM"]} <br>
            Drop: {row["Load Dropoff Date"]}
            """

            folium.PolyLine(row["Routes"]['route'],weight=8,color='black',opacity=0.6).add_to(m_od)
            if num == 1:
                folium.Marker(location=row["Deadhead_Start"]['start_point'],
                              popup = "Domicile",
                              icon=folium.Icon(icon='star', color='black')).add_to(m_od)

            ### put the deadhead lines in
            folium.PolyLine(row["Deadhead_Start"]['route'],weight=8,color='red',opacity=0.6).add_to(m_od)
            folium.PolyLine(row["Deadhead_End"]['route'],weight=8,color='red',opacity=0.6).add_to(m_od)

            folium.Marker(
                    location=row["Routes"]['start_point'],
                    icon=folium.Icon(color='green',icon_color='white'),
                    markerColor="green",
                ).add_to(m_od)
            folium.Marker(
                    location=row["Routes"]['start_point'],
                    popup=popup_pick,
                    icon= number_DivIcon(num)
                ).add_to(m_od)
            folium.Marker(
                    location=row["Routes"]['end_point'],
                    icon=folium.Icon(color='red',icon_color='white'),
                    markerColor="green",
                ).add_to(m_od)
            folium.Marker(
                    location=row["Routes"]['end_point'],
                    popup=popup_drop,
                    icon= number_DivIcon(num)
                ).add_to(m_od)
            num +=1
            box.append(row["Routes"]['route'])



        #folium.Marker(location=route['start_point'],icon=folium.Icon(icon='btc', prefix='fa', color = 'green')).add_to(m)
        #folium.Marker(location=route['end_point'],icon=folium.Icon(icon='stop', color='red')).add_to(m)

        box = [item for sublist in box for item in sublist]
        sw = [min([r[0] for r in box]),min([r[1] for r in box])]
        ne = [max([r[0] for r in box]),max([r[1] for r in box])]

        m_od.fit_bounds([sw, ne]) 
        #m_od


        ##### SAVE
        iteration_dict["Map OD Route"] = m_od




        #### loads employed
        #cycle
        iteration_dict["OD cycle"] = cycle

        driver_constraints = constraint_data[constraint_data["DRIVER_NAME"] == cycle.iloc[0]["Driver Name"]]
        date_match = pd.to_datetime(cycle["Driver Start Move Date"].iloc[0]).date()
        driver_constraints["tour_start_difference"] = date_match - driver_constraints["ACTIVE_DRIVER_IN_PROGRAM"]
        driver_constraints["tour_start_difference_absolute"] =  abs(driver_constraints["tour_start_difference"])
        driver_constraints = driver_constraints.sort_values("tour_start_difference_absolute")

        date_match = pd.to_datetime(cycle["Load Dropoff Date"].iloc[-1]).date()
        driver_constraints["tour_end_difference"] = date_match - driver_constraints["DUE_HOME_ON"]
        driver_constraints["tour_end_difference_absolute"] =  abs(driver_constraints["tour_end_difference"])


        iteration_dict["Driver Constraints"] = driver_constraints
        print(f"Fleet program is {driver_constraints.Program.iloc[0]}")
        #driver_constraints.head()

        ### what constraints match up in the OD file
        od_cycle = OD_low_conf_routes[OD_low_conf_routes["Tour ID"] == tours[tour_id]]
        iteration_dict["OD Cycle From Their Data"] = od_cycle
        #od_cycle


        driver_constraints_OD = OD_constraints[OD_constraints["Driver_Hidden"] == od_cycle["Driver ID"].iloc[0]]
        driver_constraints_OD["ACTIVE_DRIVER_IN_PROGRAM"] = driver_constraints_OD["ACTIVE_DRIVER_IN_PROGRAM"].apply(lambda x: pd.to_datetime(x).date())
        date_match = pd.to_datetime(od_cycle["Driver Start Move Date"].iloc[0]).date()
        driver_constraints_OD["tour_start_difference"] = date_match - driver_constraints_OD["ACTIVE_DRIVER_IN_PROGRAM"]

        driver_constraints_OD["tour_start_difference"] = date_match - driver_constraints_OD["ACTIVE_DRIVER_IN_PROGRAM"]
        driver_constraints_OD["tour_start_difference_absolute"] =  abs(driver_constraints_OD["tour_start_difference"])

        driver_constraints_OD = driver_constraints_OD.sort_values("tour_start_difference_absolute")

        iteration_dict["OD Driver Constraints from Their File"] = driver_constraints_OD
        #driver_constraints_OD.head()

        ##############
        ##############
        ##############
        ### what loads did we run for this driver for this tour?
        our_cycle = data[data.DRIVER_FIRST_NAME ==  cycle.iloc[0]["Driver Name"]]
        s = our_cycle["LOAD_ACTIVITY_DATE"] >= driver_constraints.ACTIVE_DRIVER_IN_PROGRAM.iloc[0]
        e = our_cycle["LAST_DROP_ARRIVAL_DATETIME"].dt.date <= driver_constraints.DUE_HOME_ON.iloc[0]
        our_cycle = our_cycle[(s & e)]
        our_cycle = our_cycle.sort_values("LOAD_ACTIVITY_DATE")

        if our_cycle.shape[0] > 0:

            ### lets chain these loads together
            Driver_Starting_City_State = list()
            Driver_Ending_City_State = list()
            ctr = 0
            for k,v in our_cycle.iterrows():
                if ctr == 0:
                    Driver_Starting_City_State.append(driver_constraints.iloc[0]["HOME_TIME_CITY"] + "," + driver_constraints.iloc[0]["HOME_TIME_STATE"])
                else:
                    Driver_Starting_City_State.append(our_cycle.iloc[ctr-1]["DESTINATION_CITY"] + "," + our_cycle.iloc[ctr-1]["DESTINATION_STATE"])

                ctr+=1
                if ctr == (our_cycle.shape[0]):
                    Driver_Ending_City_State.append(driver_constraints.iloc[0]["HOME_TIME_CITY"] + "," + driver_constraints.iloc[0]["HOME_TIME_STATE"])
                else:
                    Driver_Ending_City_State.append(our_cycle.iloc[ctr-1]["DESTINATION_CITY"] + "," + our_cycle.iloc[ctr-1]["DESTINATION_STATE"])
                    pass





            our_cycle["Driver Starting City"] = Driver_Starting_City_State
            our_cycle["Driver Ending City"] = Driver_Ending_City_State

            ##### here is the robinson map
            ##### Look into this...
            #cycle = OD_low_conf_routes[OD_low_conf_routes["Tour ID"] == tours[tour_id]]

            #cycle = cycle.merge(data, how = "left", left_on = "Load ID", right_on = "LOAD_NUM")

            #cycle = cycle.sort_values("LOAD_ACTIVITY_DATE")
            cycle = our_cycle



            cycle['origin_lat'] = cycle.apply(lambda row: get_lat(row['ORIGIN_CITY'], row['ORIGIN_STATE']), axis=1)
            cycle['origin_lng'] = cycle.apply(lambda row: get_lng(row['ORIGIN_CITY'], row['ORIGIN_STATE']), axis=1)
            cycle['destination_lat'] = cycle.apply(lambda row: get_lat(row['DESTINATION_CITY'], row['DESTINATION_STATE']), axis=1)
            cycle['destination_lng'] = cycle.apply(lambda row: get_lng(row['DESTINATION_CITY'], row['DESTINATION_STATE']), axis=1)


            #### do the start finish provided by OD #####
            cycle['driver_start_lat'] = cycle.apply(lambda row: get_lat(row["Driver Starting City"
                                                                           ].split(",")[0], row["Driver Starting City"
                                                                                               ].split(",")[1]), axis=1)
            cycle['driver_start_lng'] = cycle.apply(lambda row: get_lng(row["Driver Starting City"
                                                                           ].split(",")[0], row["Driver Starting City"
                                                                                               ].split(",")[1]), axis=1)

            cycle['driver_end_lat'] = cycle.apply(lambda row: get_lat(row["Driver Ending City"
                                                                           ].split(",")[0], row["Driver Ending City"
                                                                                               ].split(",")[1]), axis=1)
            cycle['driver_end_lng'] = cycle.apply(lambda row: get_lng(row["Driver Ending City"
                                                                           ].split(",")[0], row["Driver Ending City"
                                                                                               ].split(",")[1]), axis=1)


            ### get the routes

            plotting = list()
            route_keys = list()
            deadhead_start = list()
            deadhead_end = list()
            for k,row in cycle.iterrows():
                osrm_results = get_o_to_d(row['origin_lat'], row['origin_lng'], row['destination_lat'], row['destination_lng'])
                encoded_polyline = osrm_results["routes"][0]["geometry"]

                routes = polyline.decode(encoded_polyline)
                start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
                end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
                distance = osrm_results['routes'][0]['distance']

                route = {'route':routes,
                           'start_point':start_point,
                           'end_point':end_point,
                           'distance':(distance/1609.344)
                          }
                plotting.append(route)

                #### deadhead lines for starting city
                osrm_results = get_o_to_d(row['driver_start_lat'], row['driver_start_lng'], row['origin_lat'], row['origin_lng'])
                encoded_polyline = osrm_results["routes"][0]["geometry"]

                routes = polyline.decode(encoded_polyline)
                start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
                end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
                distance = osrm_results['routes'][0]['distance']

                route = {'route':routes,
                           'start_point':start_point,
                           'end_point':end_point,
                           'distance':(distance/1609.344)
                          }
                deadhead_start.append(route)


                #### deadhead lines for ending city
                osrm_results = get_o_to_d(row['destination_lat'], row['destination_lng'], row['driver_end_lat'], row['driver_end_lng'])
                encoded_polyline = osrm_results["routes"][0]["geometry"]

                routes = polyline.decode(encoded_polyline)
                start_point = [osrm_results['waypoints'][0]['location'][1], osrm_results['waypoints'][0]['location'][0]]
                end_point = [osrm_results['waypoints'][1]['location'][1], osrm_results['waypoints'][1]['location'][0]]
                distance = osrm_results['routes'][0]['distance']

                route = {'route':routes,
                           'start_point':start_point,
                           'end_point':end_point,
                           'distance':(distance/1609.344)
                          }
                deadhead_end.append(route)


                route_keys.append(k)



            cycle["Routes"] = plotting
            cycle["Deadhead_Start"] = deadhead_start
            cycle["Deadhead_End"] = deadhead_end

            ### make the f-ing map
            m = folium.Map()
            num = 1
            box=list()
            for k,row in cycle.iterrows():
                popup_pick = f"""L{row["LOAD_NUM"]} <br>
                Pick Ready: {row["PICK_READY_DATE"].strftime("%m/%d")}
                Pick By: {row["PICK_BY_DATE"].strftime("%m/%d")}
                """

                popup_drop = f"""L{row["LOAD_NUM"]} <br>
                Drop: {row["DROP_BY_DATE"]}
                """

                folium.PolyLine(row["Routes"]['route'],weight=8,color='black',opacity=0.6).add_to(m)
                if num == 1:
                    folium.Marker(location=row["Deadhead_Start"]['start_point'],
                                  popup = "Domicile",
                                  icon=folium.Icon(icon='star', color='black')).add_to(m)

                ### put the deadhead lines in
                folium.PolyLine(row["Deadhead_Start"]['route'],weight=8,color='red',opacity=0.6).add_to(m)
                folium.PolyLine(row["Deadhead_End"]['route'],weight=8,color='red',opacity=0.6).add_to(m)

                folium.Marker(
                        location=row["Routes"]['start_point'],
                        icon=folium.Icon(color='green',icon_color='white'),
                        markerColor="green",
                    ).add_to(m)
                folium.Marker(
                        location=row["Routes"]['start_point'],
                        popup=popup_pick,
                        icon= number_DivIcon(num)
                    ).add_to(m)
                folium.Marker(
                        location=row["Routes"]['end_point'],
                        icon=folium.Icon(color='red',icon_color='white'),
                        markerColor="green",
                    ).add_to(m)
                folium.Marker(
                        location=row["Routes"]['end_point'],
                        popup=popup_drop,
                        icon= number_DivIcon(num)
                    ).add_to(m)
                num +=1
                box.append(row["Routes"]['route'])



            #folium.Marker(location=route['start_point'],icon=folium.Icon(icon='btc', prefix='fa', color = 'green')).add_to(m)
            #folium.Marker(location=route['end_point'],icon=folium.Icon(icon='stop', color='red')).add_to(m)

            box = [item for sublist in box for item in sublist]
            sw = [min([r[0] for r in box]),min([r[1] for r in box])]
            ne = [max([r[0] for r in box]),max([r[1] for r in box])]

            m.fit_bounds([sw, ne]) 

            ### save
            iteration_dict["Map Robinson Route"] = m

            iteration_dict["Robinson Planned Cycle"] = cycle
        #### save it all
        Loop_output.append(iteration_dict)
    except:
        print("Failed Loop")

In [ ]:
tour_id_lst = list()
days_in_program_lst = list()
miles_lst = list()
revenue_lst = list()
fixed_cost_lst = list()
variable_cost_lst = list()
total_cost_lst = list()
agp_lst = list()
broker_agp_lst = list()
program_lst = list()
n_lst = list()


tmp = [43] 
#tmp = range(len(Loop_output))
for Loop_number in tmp:

    #Loop_number = 6 ### here is all you need to control to iterate.

    driver_tour_constraints = Loop_output[Loop_number]['Driver Constraints'].iloc[0]

    driver = Loop_output[Loop_number]['Driver Constraints'].DRIVER_NAME.iloc[0]

    fleet = Loop_output[Loop_number]['Driver Constraints'].FLEET_NAME.iloc[0]

    fleet_hidden = Loop_output[Loop_number]['Driver Constraints'].Fleet_Hidden.iloc[0]

    driver_equipment = data[data.DRIVER_FIRST_NAME == driver].EXECUTION_SERVICE_NAME.mode()[0]

    program = Loop_output[Loop_number]['Driver Constraints'].Program.iloc[0]




    cycle = Loop_output[Loop_number]['OD cycle']

    driver_hidden = Loop_output[Loop_number]['OD cycle']['Driver ID'].iloc[0]

    tour_id = Loop_output[Loop_number]['OD cycle']['Tour ID'].iloc[0]



    miles = 0
    for k,v in cycle.iterrows(): 
        miles += v['Routes']['distance']
        miles += v['Deadhead_Start']['distance']
        miles += v['Deadhead_End']['distance']
    miles

    dys = (driver_tour_constraints.DUE_HOME_ON - driver_tour_constraints.ACTIVE_DRIVER_IN_PROGRAM).days + 1

    od_dys = (pd.to_datetime(cycle["Load Dropoff Date"].iloc[-1]) - pd.to_datetime(cycle["Driver Start Move Date"].iloc[0])).days +1

    committed_days = (max(driver_tour_constraints.DUE_HOME_ON,
                        pd.to_datetime(cycle["Load Dropoff Date"].iloc[-1]).date()) - min(
                        driver_tour_constraints.ACTIVE_DRIVER_IN_PROGRAM,
                        pd.to_datetime(cycle["Driver Start Move Date"].iloc[0]).date())
                     ).days + 1


    if program != '7% of Revenue to Robinson':

        money = rates[rates["Fleet"] == driver_tour_constraints.FLEET_NAME]
        money = money[money["Date of Change"].dt.date <= driver_tour_constraints.ACTIVE_DRIVER_IN_PROGRAM]
        money = money.sort_values("Date of Change", ascending=False).iloc[0]

        DOW = dict()
        for i in pd.date_range(start='1/1/2018', periods=7):
            if i.strftime("%a") in ["Mon","Tue","Wed","Thu","Fri"]:
                DOW[i.strftime("%a")] = money["M-F Daily Rate"]
            if i.strftime("%a") in ["Sat"]:
                DOW[i.strftime("%a")] = money["Saturday Rate"]
            if i.strftime("%a") in ["Sun"]:
                DOW[i.strftime("%a")] = money["Sunday Rate"]


        start =min(
                    driver_tour_constraints.ACTIVE_DRIVER_IN_PROGRAM,
                    pd.to_datetime(cycle["Driver Start Move Date"].iloc[0]).date()
        )



        end = max(driver_tour_constraints.DUE_HOME_ON,
                            pd.to_datetime(cycle["Load Dropoff Date"].iloc[-1]).date())

        costs = list()
        DOW_PGM = list()
        for i in pd.date_range(start,end):
            DOW_PGM.append(i.strftime("%a"))
            costs.append(DOW[i.strftime("%a")])

        fixed_cost = np.nansum(costs)


        cost_basis = fixed_cost + driver_tour_constraints.Fuel_Surcharge_Per_Mile*miles

    else:

        fixed_cost = 0 
        cost_basis = 0.93*cycle.CustomerRate.sum()
        DOW_PGM = []
        costs = []




    cycle.rename(columns={"EXECUTION_SERVICE_NAME":"Mode",
                         "ADJUSTED_GROSS_PROFIT":"FileAvg"}, inplace=True)
    
    tour_id_lst.append(cycle["Tour ID"].iloc[0])
    days_in_program_lst.append(committed_days)
    miles_lst.append(int(miles))
    revenue_lst.append(int(cycle.CustomerRate.sum()))
    fixed_cost_lst.append(int(fixed_cost))
    try:
        variable_cost_lst.append(int(driver_tour_constraints.Fuel_Surcharge_Per_Mile*miles))
    except:
        variable_cost_lst.append(0)
    total_cost_lst.append(int(cost_basis))
    agp_lst.append(int(cycle.CustomerRate.sum() - cost_basis))
    broker_agp_lst.append(int(cycle.FileAvg.sum()))
    program_lst.append(program)
    n_lst.append(cycle.shape[0])


msg = f"""
----------------- Optimal Dynamics Route -------------------
Driver ID: {driver_hidden} Tour ID: {tour_id}
Driver Equipment Type: {driver_equipment}
Tour Miles: {miles:0.0f} Tour Revenue: ${cycle.CustomerRate.sum():0.2f}
Days in Program: {committed_days} days {DOW_PGM}
Fixed Daily Amounts: {costs}
Tour Cost Basis Paid to Fleet {program}: ${cost_basis:0.2f}
Fixed: ${fixed_cost:0.2f} Variable: ${driver_tour_constraints.Fuel_Surcharge_Per_Mile*miles:0.2f}
Dedicated AGP On Tour:  ${cycle.CustomerRate.sum() - cost_basis:0.2f}
Brokerage AGP if Executed Traditionally: ${cycle.FileAvg.sum():0.2f} 
AGP if we used 7% program: ${cycle.CustomerRate.sum()*0.07:0.2f}
-----------------------------------------------------
Driver Start and End VS Constraints:
{driver_tour_constraints[["ACTIVE_DRIVER_IN_PROGRAM",
                        "DUE_HOME_ON",
                        "tour_start_difference",
                        "tour_end_difference"]]}
-----------------------------------------------------
"""
print(msg)
display(HTML(cycle[["LOAD_NUM",
                   "Mode",
                    "LOADING_TYPE",
                    "Load Pickup Date",
                   "Load Dropoff Date",
                   "CustomerRate",
                    "FileAvg"]].to_html()))
Loop_output[Loop_number]['Map OD Route']